In [ ]:
# AMPL - algebraic modeling programming language
# Dinner Seating Plan - Tables
# 6 families, 5 tables
# max 3 members of family at 1 table

In [14]:
# install dependencies and select solver
%pip install -q amplpy pandas networkx

SOLVER = "cbc"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cbc"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


In [15]:
from IPython.display import display
import networkx as nx
import pandas as pd

In [16]:
%%writefile table_seat.mod

set F;
set T;

param M{F};
param C{T};

param k;

var x{F, T} >= 0, <= k;
var dummy >= 0, <= 1, default 0;

minimize goal: dummy;

s.t. capacity {t in T}: sum{f in F} x[f, t] <= C[t];
s.t. seat {f in F}: sum{t in T} x[f, t] == M[f];

Writing table_seat.mod


In [17]:
def table_seat(members, capacity, k):
    m = AMPL()
    m.read("table_seat.mod")

    m.set["F"] = range(len(members))
    m.set["T"] = range(len(capacity))

    m.param["M"] = members
    m.param["C"] = capacity

    m.param["k"] = k

    m.option["solver"] = SOLVER

    return m


def get_solution(model):
    df = model.var["x"].to_pandas()
    df.index.names = ["family", "table"]
    df = df.unstack()
    df.columns = df.columns.get_level_values(1)

    return df.round(5)


def report(model, results, type=int):
    print(f"Solve result: {model.solve_result}")
    if model.solve_result == "solved":
        soln = get_solution(model).astype(type)
        display(soln)
        print(f'objective:       {model.obj["goal"].value()}')
        print(f"places at table: {list(soln.sum(axis=0))}")
        print(f"members seated:  {list(soln.sum(axis=1))}")

In [18]:
# max 3 members of family at one table
seatplan = table_seat(members=[6, 8, 2, 9, 13, 1], capacity=[8, 8, 10, 4, 9], k=3)
%time results = seatplan.solve()
assert seatplan.solve_result == "solved", seatplan.solve_result
report(seatplan, results, type=float)

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 0
0 simplex iterations
CPU times: user 3.4 ms, sys: 61 µs, total: 3.46 ms
Wall time: 26.2 ms
Solve result: solved


table,0,1,2,3,4
family,,,,,
0,0.0,3.0,1.0,0.0,2.0
1,3.0,0.0,3.0,0.0,2.0
2,0.0,0.0,0.0,0.0,2.0
3,1.0,2.0,3.0,3.0,0.0
4,3.0,3.0,3.0,1.0,3.0
5,1.0,0.0,0.0,0.0,0.0


objective:       0.0
places at table: [8.0, 8.0, 10.0, 4.0, 9.0]
members seated:  [6.0, 8.0, 2.0, 9.0, 13.0, 1.0]


In [20]:
# replace treshhold 3 by more precise treshhold
# find more precise treshhold
%%writefile table_seat_minimize_max_group_at_table.mod

set F;
set T;

param M{F};
param C{T};

var x{F,T} >= 0;
var k >= 0;

minimize goal: k;

s.t. capacity {t in T}: sum{f in F} x[f,t] <= C[t];
s.t. seat {f in F}: sum{t in T} x[f,t] == M[f];
s.t. bound {f in F, t in T}: x[f,t] <= k;

Overwriting table_seat_minimize_max_group_at_table.mod


In [21]:
def table_seat_minimize_max_group_at_table(members, capacity):
    m = AMPL()
    m.read("table_seat_minimize_max_group_at_table.mod")

    m.set["F"] = range(len(members))
    m.set["T"] = range(len(capacity))

    m.param["M"] = members
    m.param["C"] = capacity

    m.option["solver"] = SOLVER

    return m

In [23]:
# it computed relative treshhold 2.6
seatplan = table_seat_minimize_max_group_at_table(
    members=[6, 8, 2, 9, 13, 1], capacity=[8, 8, 10, 4, 9]
)
%time results = seatplan.solve()
report(seatplan, results, type=float)

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 2.6
0 simplex iterations
CPU times: user 4.41 ms, sys: 0 ns, total: 4.41 ms
Wall time: 27.4 ms
Solve result: solved


table,0,1,2,3,4
family,,,,,
0,0.0,2.6,0.8,0.0,2.6
1,2.6,0.2,2.6,0.0,2.6
2,0.0,0.0,0.8,0.0,1.2
3,2.6,2.6,2.4,1.4,0.0
4,2.6,2.6,2.6,2.6,2.6
5,0.2,0.0,0.8,0.0,0.0


objective:       2.6
places at table: [8.0, 8.0, 10.0, 4.0, 9.0]
members seated:  [6.0, 8.0, 2.0, 9.0, 13.0, 1.0]


In [24]:
# minimize number of tables
%%writefile table_seat_minimize_number_of_tables.mod

set F;
set T;

param M{F};
param C{T};

param k;

var x{F,T} integer >= 0, <= k;
var y{T} binary;

minimize goal: sum{t in T} y[t];

s.t. capacity{t in T}: sum{f in F} x[f,t] <= C[t] * y[t];
s.t. seat{f in F}: sum{t in T} x[f,t] == M[f];

Writing table_seat_minimize_number_of_tables.mod


In [25]:
def table_seat_minimize_number_of_tables(members, capacity, k, relax=False):
    m = AMPL()
    m.read("table_seat_minimize_number_of_tables.mod")

    m.set["F"] = range(len(members))
    m.set["T"] = range(len(capacity))

    m.param["M"] = members
    m.param["C"] = capacity
    m.param["k"] = k

    m.option["solver"] = SOLVER

    if relax:
        m.option["relax_integrality"] = 1

    return m

In [26]:
members = [6, 8, 2, 9, 13, 1]
capacity = [8, 8, 10, 4, 9]
k = 3

print("\nMILO problem\n")
seatplan = table_seat_minimize_number_of_tables(members, capacity, k)
%time results = seatplan.solve()
report(seatplan, results, type=int)

print("\nRelaxed problem\n")
seatplan = table_seat_minimize_number_of_tables(members, capacity, k, relax=True)
%time results = seatplan.solve()
report(seatplan, results, type=float)


MILO problem

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 5
0 simplex iterations
CPU times: user 2.93 ms, sys: 836 µs, total: 3.76 ms
Wall time: 27.9 ms
Solve result: solved


table,0,1,2,3,4
family,,,,,
0,2,0,3,0,1
1,0,2,3,0,3
2,0,0,0,2,0
3,3,3,1,1,1
4,3,3,3,1,3
5,0,0,0,0,1


objective:       5.0
places at table: [8, 8, 10, 4, 9]
members seated:  [6, 8, 2, 9, 13, 1]

Relaxed problem

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 5
0 simplex iterations
CPU times: user 3.6 ms, sys: 0 ns, total: 3.6 ms
Wall time: 25.9 ms
Solve result: solved


table,0,1,2,3,4
family,,,,,
0,0.0,2.0,1.0,0.0,3.0
1,0.0,2.0,3.0,0.0,3.0
2,1.0,0.0,0.0,1.0,0.0
3,3.0,3.0,3.0,0.0,0.0
4,3.0,1.0,3.0,3.0,3.0
5,1.0,0.0,0.0,0.0,0.0


objective:       5.0
places at table: [8.0, 8.0, 10.0, 4.0, 9.0]
members seated:  [6.0, 8.0, 2.0, 9.0, 13.0, 1.0]


In [ ]:
# https://ampl.com/mo-book/notebooks/04/dinner-seat-allocation.html